In [2]:
import concurrent.futures
import json
import subprocess
import tempfile

import polars as pl

In [3]:
df = pl.concat(
    [
        pl.read_csv("optimized-sequences.csv"),
        pl.read_csv("output-hydrated.csv"),
    ],
    how="align",
)
df

index,source,name,raw_input_sequence,input_sequence,error,optimized_sequence,optimization_error,input_a,input_c,input_g,input_t,input_gc,input_gc1,input_gc2,input_gc3,input_cpg,input_uridine_depletion,input_cai,input_tai,input_mfe,input_amfe,input_mfe_structure,input_wmfe,input_gini,input_slippery_site_ratio,input_rscu,input_rcbs,input_dcbs,input_rcr,input_cub,input_cbi,optimized_a,optimized_c,optimized_g,optimized_t,optimized_gc,optimized_gc1,optimized_gc2,optimized_gc3,optimized_cpg,optimized_uridine_depletion,optimized_cai,optimized_tai,optimized_mfe,optimized_amfe,optimized_mfe_structure,optimized_wmfe,optimized_gini,optimized_slippery_site_ratio,optimized_rscu,optimized_rcbs,optimized_dcbs,optimized_rcr,optimized_cub,optimized_cbi
i64,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,"""antibody_monoclonal_aa_seqs.fa…","""RituximabTargetAnti-CD20v_heav…","""QVQLQQPGAELVKPGASVKMSCKASGYTFT…","""CAGGTGCAGCTGCAGCAGCCCGGCGCCGAG…",null,"""CAGGTGCAGCTCCAGCAGCCAGGCGCCGAA…",null,0.233555,0.350333,0.302291,0.113821,0.652624,0.652624,0.465632,0.982262,0.060606,0.0,1.0,0.372443,-522.0,-0.385809,"""(((((.(.((((...))))..(((((((.(…",-8.831611,0.163581,0.0,3.567627,0.815107,2.385627,0.0,1.093384,1.0,0.232077,0.343681,0.288987,0.135255,0.632668,0.632668,0.465632,0.922395,0.055432,0.042129,0.979129,0.369046,-507.799988,-0.375314,"""((((.(((((((.....((..(((((((..…",-8.361398,0.256065,0.003695,2.986456,0.750011,2.330087,0.007391,0.929305,0.873899
1,"""antibody_monoclonal_aa_seqs.fa…","""RituximabTargetAnti-CD20v_ligh…","""QIVLSQSPAILSASPGEKVTMTCRASSSVS…","""CAGATCGTGCTGAGCCAGAGCCCCGCCATC…",null,"""CAGATCGTGCTGAGCCAGAGCCCCGCCATC…",null,0.239437,0.348983,0.303599,0.107981,0.652582,0.652582,0.507042,0.971831,0.059468,0.0,1.0,0.362417,-227.5,-0.356025,"""......(((((..((....)).(((((.((…",-8.532667,0.167019,0.0,3.71831,0.856337,2.440252,0.0,1.11896,1.0,0.231612,0.350548,0.29734,0.120501,0.647887,0.647887,0.507042,0.957746,0.059468,0.014085,0.99444,0.3632,-234.399994,-0.366823,"""......(((((..(((..........((.(…",-8.714667,0.205446,0.0,3.325244,0.797784,2.489732,0.0,1.034453,0.936471
2,"""antibody_monoclonal_aa_seqs.fa…","""TrastuzumabTargetAnti-HER2_Lig…","""DIQMTQSPSSLSASVGDRVTITCRASQDVN…","""GACATCCAGATGACCCAGAGCCCCAGCAGC…",null,"""GACATCCAGATGACCCAGAGCCCCAGCTCC…",null,0.244548,0.353583,0.28972,0.11215,0.643302,0.643302,0.476636,0.96729,0.05296,0.0,1.0,0.365163,-225.199997,-0.350779,"""..(((.....((.(.(((.(((((((((((…",-8.297351,0.166355,0.0,3.668224,0.893168,2.538889,0.0,1.112429,1.0,0.238318,0.35514,0.283489,0.123053,0.638629,0.638629,0.476636,0.953271,0.05296,0.014019,0.9951,0.364569,-227.199997,-0.353894,""".......((((......((((....))))(…",-8.364901,0.20729,0.001558,3.328629,0.826603,2.576876,0.003115,1.028316,0.936657
3,"""antibody_monoclonal_aa_seqs.fa…","""TrastuzumabTargetAnti-HER2_Hea…","""EVQLVESGGGLVQPGGSLRLSCAASGFNIK…","""GAGGTGCAGCTGGTGGAGAGCGGCGGCGGC…",null,"""GAGGTGCAGCTGGTGGAGAGCGGCGGCGGC…",null,0.232593,0.341481,0.308889,0.117037,0.65037,0.65037,0.462222,0.971111,0.055556,0.0,1.0,0.369346,-525.900024,-0.389556,""".(((((....(((((..((((.((((((((…",-8.746341,0.153111,0.0,3.593333,0.83995,2.454101,0.0,1.098031,1.0,0.231852,0.334074,0.304444,0.12963,0.638519,0.638519,0.462222,0.933333,0.051111,0.028889,0.984833,0.367733,-533.400024,-0.395111,"""........(((((.((.((((.((((((((…",-8.753963,0.261373,0.000741,3.114534,0.728153,2.439528,0.003704,0.969142,0.901277
4,"""antibody_monoclonal_aa_seqs.fa…","""Bevacizumab_light_chain""","""DIQMTQSPSSLSASVGDRVTITCSASQDIS…","""GACATCCAGATGACCCAGAGCCCCAGCAGC…",null,"""GACATCCAGATGACCCAGAGCCCCAGCTCC…",null,0.246106,0.345794,0.292835,0.115265,0.638629,0.638629,0.46729,0.976636,0.051402,0.0,1.0,0.365897,-228.899994,-0.356542,"""...............(((.((.......))…",-8.643046,0.166822,0.0,3.649533,0.839185,2.424216,0.0,1.105428,1.0,0.238318,

In [4]:
def _degscore(sequence: str, structure: str) -> dict:
    with tempfile.TemporaryDirectory() as temp_dir:
        result = subprocess.run(
            [
                "docker",
                "run",
                "--volume",
                f"{temp_dir}:/output",
                "degscore:latest",
                sequence,
                structure,
                "/output/output.json",
            ],
            stdout=subprocess.DEVNULL,
            # cwd=temp_dir,
        )
        if result.returncode:
            raise RuntimeError(f"Return code: {result.returncode}")
        
        with open(temp_dir + "/output.json", "r") as f:
            return json.loads(f.read())

def _process_row(row: dict) -> dict:
    if row["index"] % 1000 == 0:
        print(f"index: {row['index']}")

    input_degscore = _degscore(
        row["input_sequence"],
        row["input_mfe_structure"]
    )
    optimized_degscore = {}
    if row["optimized_sequence"]:
        optimized_degscore = _degscore(
            row["optimized_sequence"],
            row["optimized_mfe_structure"],
        )
    return {
        "index": row["index"],
        "input_degscore": input_degscore["degscore"],
        "input_est_half_life": input_degscore["est_half_life"],
        "optimized_degscore": optimized_degscore.get("degscore"),
        "optimized_est_half_life": optimized_degscore.get("est_half_life"),
    }

with concurrent.futures.ProcessPoolExecutor() as executor:
    results = list(executor.map(
        _process_row,
        list(df.rows(named=True))
    ))

output = pl.DataFrame(results)
output.write_csv("output-degscore.csv")
output

index: 0
index: 1000
index: 2000
index: 3000
index: 4000
index: 5000
index: 6000
index: 7000
index: 8000
index: 9000
index: 10000
index: 11000
index: 12000
index: 13000
index: 14000
index: 15000
index: 16000
index: 17000
index: 18000
index: 19000
index: 20000
index: 21000
index: 22000
index: 23000
index: 24000
index: 25000
index: 26000
index: 27000
index: 28000
index: 29000
index: 30000
index: 31000
index: 32000
index: 33000
index: 34000
index: 35000
index: 36000
index: 37000
index: 38000
index: 39000
index: 40000
index: 41000
index: 42000
index: 43000
index: 44000
index: 45000
index: 46000
index: 47000
index: 48000
index: 49000
index: 50000


index,input_degscore,input_est_half_life,optimized_degscore,optimized_est_half_life
i64,f64,f64,f64,f64
0,409.421,0.736572,411.523,0.733017
1,192.913,1.471603,195.884,1.451723
2,196.898,1.445061,191.748,1.479547
3,400.314,0.752379,400.012,0.752914
4,193.698,1.466297,190.187,1.490328
…,…,…,…,…
50708,174.272,1.609925,173.675,1.614786
50709,205.05,1.393641,208.32,1.374029
50710,462.356,0.656411,464.0,0.6542
